# Fine-Tune Multi-Modal (image+text) Gemini model for Visual Q+A


CS 663: Group 6

This Colab shows how to fine-tune Gemini for the TACO dataset to categorize an image into a certain category: Plastic, Paper, Metal, Glass, or Organic.

# STEP 1: Installs (HF datasets + Vertex AI libs)

In [ ]:
#@title Install libraries (pin versions so no conflicts like script-based HF datasets load cleanly)
# --- Clean out known conflict sources (safe to skip if not present)
!pip -q uninstall -y vertexai gcsfs fsspec google-adk

# --- Pass A: HF + datasets stack (lets `datasets` pick compatible fsspec)
!pip -q install -U \
  "datasets==2.19.2" \
  "huggingface_hub>=0.24.0" \
  "tqdm>=4.66.0" \
  "Pillow>=10.4.0" \
  "ipywidgets>=8.1.2"

# --- Pass B: Google libs (no separate `vertexai` package needed)
!pip -q install -U \
  "google-genai>=0.3.0" \
  "google-cloud-aiplatform>=1.71,<2" \
  "google-cloud-storage>=2.17.0"

import datasets, fsspec, google.cloud.aiplatform as aiplatform
import vertexai  # provided by google-cloud-aiplatform
print("datasets:", datasets.__version__)
print("fsspec:", fsspec.__version__)
print("aiplatform:", aiplatform.__version__)
print("vertexai module OK")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.1/516.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.29.1 requires gcsfs!=2025.5.0,>=2023.3.0, which is not installed.
transformers 4.57.2 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.6 which is incompatible.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which i

# STEP 2: Auth + project/region + buckets



In [ ]:
#@title Authenticate and set project/region/bucket from Colab userdata (no GUI, no bucket modification)

# 🔑 Secrets expected in Colab userdata:
# - GOOGLE_CLOUD_PROJECT: Your Google Cloud project ID (Cloud Console → "Project Info")
# - GOOGLE_DEFAULT_REGION: Region for Vertex AI and storage bucket (e.g., "us-central1")
# - GOOGLE_DEFAULT_BUCKET: The exact GCS bucket to use (will NOT be modified)

# 📂 GCS_PREFIX:
# - A timestamped folder-like prefix for organizing run data within the existing bucket.

# 1) Authenticate Colab with your Google account
from google.colab import auth as colab_auth
colab_auth.authenticate_user()

# 2) Read secrets from Colab userdata
from google.colab import userdata
import datetime

PROJECT_ID = userdata.get('GOOGLE_CLOUD_PROJECT')
REGION = userdata.get('GOOGLE_DEFAULT_REGION') or "us-central1"
GCS_BUCKET = userdata.get('GOOGLE_DEFAULT_BUCKET')

assert PROJECT_ID, "❌ Missing GOOGLE_CLOUD_PROJECT in userdata."
assert GCS_BUCKET, "❌ Missing GOOGLE_DEFAULT_BUCKET in userdata."

# 3) Initialize Vertex AI
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

# 4) Create unique prefix for this run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
GCS_PREFIX = f"multimodal_sft_{timestamp}"

# 5) Helper for building gs:// URIs
def gs_uri(*parts):
    return "gs://" + "/".join([GCS_BUCKET] + [p.strip("/") for p in parts])

# 6) Save globally for reuse
globals()["PROJECT_ID"] = PROJECT_ID
globals()["REGION"] = REGION
globals()["GCS_BUCKET"] = GCS_BUCKET
globals()["GCS_PREFIX"] = GCS_PREFIX
globals()["gs_uri"] = gs_uri

# 7) Print summary
print("✅ Environment initialized")
print("✅ Project:", PROJECT_ID)
print("✅ Region:", REGION)
print("✅ Bucket:", f"gs://{GCS_BUCKET}")
print("📂 Prefix:", GCS_PREFIX)


✅ Environment initialized
✅ Project: project-2-group-6
✅ Region: us-central1
✅ Bucket: gs://my-gemini-sft-bucket-group-6
📂 Prefix: multimodal_sft_20251130_040559


# STEP 3: Choose a multimodal base model + "tuning knobs"


In [ ]:
# STEP 3: Choose base Gemini model + tuning params (with widgets)
import ipywidgets as widgets
from IPython.display import display, clear_output

# Use prior values if they exist, else defaults
_default_base   = globals().get("BASE_MODEL", "gemini-2.5-flash")
_default_epochs = int(globals().get("EPOCHS", 2))
_default_adapt  = str(globals().get("ADAPTER_SIZE", "SMALL"))
_default_lrm    = float(globals().get("LEARNING_RATE_MULTIPLIER", 1.0))

# Adapter size mapping (string -> integer bottleneck size)
_ADAPTER_TO_INT = {"TINY": 1, "SMALL": 4, "MEDIUM": 8, "LARGE": 16}

# Widgets
base_dd = widgets.Dropdown(
    options=["gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash","gemini-2.0-flash-lite"],
    value=_default_base,
    description="Base model:",
    layout=widgets.Layout(width="420px"),
)

epochs_in = widgets.IntText(
    value=_default_epochs, description="Epochs:", min=1, step=1, layout=widgets.Layout(width="180px")
)

adapter_dd = widgets.Dropdown(
    options=[("Tiny (1)", "TINY"), ("Small (4)", "SMALL"), ("Medium (8)", "MEDIUM"), ("Large (16)", "LARGE")],
    value=_default_adapt,
    description="Adapter:",
    layout=widgets.Layout(width="220px"),
)

lrm_in = widgets.FloatText(
    value=_default_lrm, description="LR Mult:", layout=widgets.Layout(width="180px")
)

apply_btn = widgets.Button(description="Apply settings", button_style="primary")
out = widgets.Output()

def on_apply(_):
    with out:
        clear_output()
        # Write selections into globals so later cells see them
        globals()["BASE_MODEL"] = base_dd.value
        globals()["EPOCHS"] = int(epochs_in.value)
        globals()["ADAPTER_SIZE"] = adapter_dd.value
        globals()["ADAPTER_SIZE_INT"] = _ADAPTER_TO_INT[adapter_dd.value]  # for APIs needing an int
        globals()["LEARNING_RATE_MULTIPLIER"] = float(lrm_in.value)

        print("✔ Settings applied")
        print("Base model:", BASE_MODEL)
        print("Epochs:", EPOCHS)
        print("Adapter:", ADAPTER_SIZE, "(int:", ADAPTER_SIZE_INT, ")")
        print("LR multiplier:", LEARNING_RATE_MULTIPLIER)

apply_btn.on_click(on_apply)

ui = widgets.VBox([
    base_dd,
    widgets.HBox([epochs_in, adapter_dd, lrm_in]),
    apply_btn,
    out
])

display(ui)


## STEP 3.1 adapter name convert to integer


In [ ]:
#STEP 3.1 — Map ADAPTER_SIZE to integer (SDK expects 1,4,8,16)
# Accepts either labels ("TINY","SMALL","MEDIUM","LARGE") or integers/strings 1/4/8/16

_ADAPTER_LABEL_TO_INT = {
    "TINY": 1, "SMALL": 4, "MEDIUM": 8, "LARGE": 16,
    "tiny": 1, "small": 4, "medium": 8, "large": 16,
}

def normalize_adapter_size(x):
    if isinstance(x, (int, float)):
        xi = int(x)
        if xi in (1, 4, 8, 16):
            return xi
    if isinstance(x, str):
        x = x.strip()
        if x.isdigit() and int(x) in (1, 4, 8, 16):
            return int(x)
        if x in _ADAPTER_LABEL_TO_INT:
            return _ADAPTER_LABEL_TO_INT[x]
    raise ValueError("Invalid ADAPTER_SIZE. Use one of {1,4,8,16} or labels TINY/SMALL/MEDIUM/LARGE.")

ADAPTER_SIZE_NUM = normalize_adapter_size(ADAPTER_SIZE)
print("✅ Using adapter_size:", ADAPTER_SIZE_NUM)


✅ Using adapter_size: 4


# STEP 4: Import dataset from Google Drive and select train, validation, and test samples

## *Important: You must have a folder in your Google Drive called TACO where the dataset is*

Here is a link to an drive that you can copy to your own drive: https://drive.google.com/drive/u/1/folders/1YFmuOqXV_8k6Srm4_2oEVKblsWaNWg3a



In [ ]:
from google.colab import drive
import os, json
import pandas as pd
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output

# -----------------------------
# Configure sample sizes here
N_TRAIN = 15
N_VAL   = 10
N_TEST  = 10
# -----------------------------

# Mount Google Drive
drive.mount('/content/drive')

# Path to your TACO images folder
IMAGES_DIR = "/content/drive/MyDrive/TACO"

answers_path = os.path.join(IMAGES_DIR, "answers.json")
with open(answers_path, "r") as f:
    answers = json.load(f)

# Build DataFrame with image paths
image_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(('.jpg','.png'))])
records = []
for f in image_files:
    path = os.path.join(IMAGES_DIR, f)
    answer = answers.get(f, "Unknown")
    try:
        img = Image.open(path).convert("RGB")
        records.append({
            "image": img,
            "question": "What can the objects in this image be reused for?",
            "answer": answer
        })
    except Exception as e:
        print(f"Skipping {f}: {e}")

df = pd.DataFrame(records)
print(f"Loaded {len(df)} images")

# -----------------------------
# Widget setup
shuffle_dd = widgets.Dropdown(options=['Yes', 'No'], value='Yes', description='Shuffle?')
apply_btn  = widgets.Button(description='Apply', button_style='primary')
out        = widgets.Output()

def _normalize_schema(df):
    # Already has 'question', 'answer', 'image'
    return df[['question', 'answer', 'image']].reset_index(drop=True)

def _process(shuffle_choice: str):
    # Shuffle if requested
    data = df.sample(frac=1, random_state=42).reset_index(drop=True) if shuffle_choice == 'Yes' else df.copy()
    total = len(data)

    n_train = min(N_TRAIN, total)
    n_val   = min(N_VAL, total - n_train)
    n_test  = min(N_TEST, total - n_train - n_val)

    ds_train = _normalize_schema(data.iloc[:n_train])
    ds_val   = _normalize_schema(data.iloc[n_train:n_train+n_val])
    ds_test  = _normalize_schema(data.iloc[n_train+n_val:n_train+n_val+n_test])

    globals()['ds_train'] = ds_train
    globals()['ds_val']   = ds_val
    globals()['ds_test']  = ds_test

    return ds_train, ds_val, ds_test

def _on_apply(_):
    with out:
        clear_output(wait=True)
        t, v, u = _process(shuffle_dd.value)
        globals()['ds'] = {
            "train": globals()['ds_train'],
            "validation": globals()['ds_val'],
            "test": globals()['ds_test']
        }
        print(f"Done. Shuffle: {shuffle_dd.value}")
        print(f"ds_train: {len(t)} rows")
        print(f"ds_val:   {len(v)} rows")
        print(f"ds_test:  {len(u)} rows")

apply_btn.on_click(_on_apply)
display(widgets.VBox([shuffle_dd, apply_btn, out]))


Mounted at /content/drive
Loaded 35 images


Display a random image

In [ ]:
from IPython.display import display

# display a random training example
print('QUESTION:', ds_train.iloc[4]['question'])
display(ds_train.iloc[4]['image'])
print('ANSWER:', ds_train.iloc[4]['answer'])


# STEP 5: Build Gemini SFT JSONL (image + question → answer) and upload to GCS




In [ ]:
# Build Gemini SFT JSONL (robust row handling) and upload to GCS
import os, io, json, pathlib, tempfile, mimetypes, requests
from typing import Any, Dict, List, Tuple, Optional
from PIL import Image
from tqdm import tqdm
from google.cloud import storage
import pandas as pd

# -------------------------
# Preconditions / globals
# -------------------------
assert 'ds' in globals(), "Run Step 4 to load a dataset into `ds` first."
assert 'GCS_BUCKET' in globals() and 'PROJECT_ID' in globals(), "Run the auth/setup cell first."

# SELECTED_HF_NAME should be set in Step 4; derive a fallback if not.
if 'SELECTED_HF_NAME' not in globals() or not SELECTED_HF_NAME:
    SELECTED_HF_NAME = (
        getattr(ds, 'builder_name', None)
        or getattr(getattr(next(iter(ds.values())), 'info', None), 'builder_name', None)
        or "dataset"
    )

# TRAIN/VAL split names: respect existing globals; otherwise pick from ds keys
available_splits = list(ds.keys())
def _pick_train(keys):
    for k in ("train","training","Train","TRAIN"):
        if k in keys: return k
    return keys[0] if keys else "train"
def _pick_val(keys):
    for k in ("validation","val","dev","test","Test","TEST"):
        if k in keys: return k
    return "train"

if 'TRAIN_SPLIT' not in globals(): TRAIN_SPLIT = _pick_train(available_splits)
if 'VAL_SPLIT'   not in globals(): VAL_SPLIT   = _pick_val(available_splits)

# Example caps: use UI-provided N_TRAIN/N_VAL if present; else safe fallbacks
MAX_TRAIN_EXAMPLES = globals().get('N_TRAIN', 1000)
MAX_VAL_EXAMPLES   = globals().get('N_VAL',   200)

# GCS object prefix for this run (if not set earlier)
from datetime import datetime
if 'GCS_PREFIX' not in globals() or not GCS_PREFIX:
    GCS_PREFIX = f"gemini_sft/{datetime.now().strftime('%Y%m%d-%H%M%S')}"

print(f"Selected HF dataset: {SELECTED_HF_NAME}")
print(f"Splits: train='{TRAIN_SPLIT}'  val='{VAL_SPLIT}'  (available: {available_splits})")
print(f"Caps: MAX_TRAIN_EXAMPLES={MAX_TRAIN_EXAMPLES}  MAX_VAL_EXAMPLES={MAX_VAL_EXAMPLES}")
print(f"GCS: gs://{GCS_BUCKET}/{GCS_PREFIX}/...")

# -------------------------
# Utilities: access splits
# -------------------------
def _has(name: str) -> bool:
    return name in globals()

def _get_raw_split(name: str):
    """Return a row-indexable collection for the requested split."""
    if name == "train" and _has("ds_train"): return ds_train
    if name in ("validation", "val") and _has("ds_val"): return ds_val
    if name == "test" and _has("ds_test"): return ds_test
    if name in ds: return ds[name]
    # fallback to common alternates
    for alt in ("validation", "val", "test", "train"):
        if alt in ds: return ds[alt]
    raise ValueError(f"Split '{name}' not found. Available: {list(ds.keys())}")

def _row_count(raw) -> int:
    try:
        return len(raw)
    except Exception:
        if isinstance(raw, dict) and raw:
            try:
                return min(len(v) for v in raw.values())
            except Exception:
                return 0
        return 0

def _hf_dataset_type():
    try:
        from datasets import Dataset as HFDataset
        return HFDataset
    except Exception:
        return tuple()

def _get_row(raw, i: int) -> Dict[str, Any]:
    """Return a dict-like example for row i across many container types."""
    HFDataset = _hf_dataset_type()

    # HF Dataset
    if HFDataset and isinstance(raw, HFDataset):
        ex = raw[int(i)]
        return ex if isinstance(ex, dict) else {"value": ex}

    # pandas DataFrame
    if isinstance(raw, pd.DataFrame):
        return raw.iloc[int(i)].to_dict()

    # list/tuple
    if isinstance(raw, (list, tuple)):
        ex = raw[int(i)]
        return ex if isinstance(ex, dict) else {"value": ex}

    # dict-of-lists
    if isinstance(raw, dict) and all(isinstance(v, list) for v in raw.values()):
        out = {}
        for k, v in raw.items():
            if len(v) > i:
                out[k] = v[int(i)]
        return out

    # generic fallback
    try:
        ex = raw[int(i)]
        return ex if isinstance(ex, dict) else {"value": ex}
    except Exception:
        return {"value": raw}

# -------------------------
# Field extraction (robust to arrays/lists)
# -------------------------
def _first_present(ex: Dict[str, Any], keys: List[str]):
    for k in keys:
        if k in ex and ex[k] is not None:
            return ex[k]
    return None

def _to_list_of_str(v: Any) -> List[str]:
    """Coerce v into a list[str]. Handles str, list/tuple, ndarray, list[dict] with 'answer'."""
    try:
        import numpy as np
    except Exception:
        np = None

    if v is None:
        return []

    if isinstance(v, str):
        return [v]

    # HF 'answers' like [{"answer": "x"}, ...]
    if isinstance(v, list) and v and isinstance(v[0], dict) and "answer" in v[0]:
        return [str(d["answer"]) for d in v if "answer" in d]

    if isinstance(v, (list, tuple)):
        return [str(x) for x in v]

    if np is not None and isinstance(v, np.ndarray):
        if v.shape == ():
            return [str(v.item())]
        return [str(x) for x in v.tolist()]

    return [str(v)]

def _get_question_list(ex: Dict[str, Any]) -> List[str]:
    raw = _first_present(ex, ["question", "question_text", "prompt", "Q"])
    return _to_list_of_str(raw)

def _get_answer_list(ex: Dict[str, Any]) -> List[str]:
    raw = _first_present(ex, ["answers", "answer", "label", "A"])
    return _to_list_of_str(raw)

def _get_image_obj(ex: Dict[str, Any]) -> Any:
    return (
        ex.get("image")
        or ex.get("image_path")
        or ex.get("image_url")
        or ex.get("img")
        or ex.get("filepath")
        or ex.get("file")
        or ex.get("path")
    )

# -------------------------
# Image → local path
# -------------------------
def _save_pil_to_tmp(img: Image.Image, suffix=".jpg") -> str:
    fp = tempfile.NamedTemporaryFile(delete=False, suffix=suffix).name
    img.save(fp, "JPEG" if suffix.lower() in (".jpg", ".jpeg") else "PNG")
    return fp

def _to_local_path(obj: Any) -> Optional[str]:
    # PIL.Image.Image
    if isinstance(obj, Image.Image):
        return _save_pil_to_tmp(obj, ".jpg")

    # dict with bytes/path/url
    if isinstance(obj, dict):
        if "bytes" in obj:
            try:
                im = Image.open(io.BytesIO(obj["bytes"]))
                return _save_pil_to_tmp(im, ".jpg")
            except Exception:
                pass
        if "path" in obj:
            return obj["path"]
        if "url" in obj:
            obj = obj["url"]  # fall through to str

    # str: URL or filesystem
    if isinstance(obj, str):
        if obj.startswith(("http://", "https://")):
            r = requests.get(obj, timeout=30)
            r.raise_for_status()
            suffix = pathlib.Path(obj).suffix or ".jpg"
            fp = tempfile.NamedTemporaryFile(delete=False, suffix=suffix).name
            with open(fp, "wb") as f:
                f.write(r.content)
            return fp
        return obj  # assume local path

    # array-like → try PIL
    try:
        im = Image.fromarray(obj)  # type: ignore[arg-type]
        return _save_pil_to_tmp(im, ".jpg")
    except Exception:
        return None

def _guess_mime(path: str) -> str:
    mt, _ = mimetypes.guess_type(path)
    return mt or "image/jpeg"

# -------------------------
# GCS upload
# -------------------------
storage_client = storage.Client(project=PROJECT_ID)
_bucket = storage_client.bucket(GCS_BUCKET)

def _upload(local_path: str, dst_key: str) -> str:
    blob = _bucket.blob(dst_key)
    blob.upload_from_filename(local_path)
    return f"gs://{GCS_BUCKET}/{dst_key}"

# -------------------------
# Collect rows for a split (fans out multi-Q/A)
# -------------------------
def _collect_for_split(split_name: str, max_n: int) -> List[Tuple[str, str, str, str]]:
    """
    Returns list of (question, answer, gs://image_uri, mime) for the split.
    If a sample has multiple Q/A (lists/ndarrays), emit one row per pair,
    reusing the same uploaded image URI.
    """
    raw = _get_raw_split(split_name)
    total = _row_count(raw)
    want = min(max_n, total if total is not None else max_n)
    rows: List[Tuple[str, str, str, str]] = []

    for i in tqdm(range(total), desc=f"Collect {split_name}"):
        if len(rows) >= want:
            break
        ex = _get_row(raw, i)

        img_obj = _get_image_obj(ex)
        if not img_obj:
            continue

        lp = _to_local_path(img_obj)
        if not lp or not pathlib.Path(lp).exists():
            continue

        mime = _guess_mime(lp)
        dst_key = f"{GCS_PREFIX}/{SELECTED_HF_NAME}/{split_name}/images/{i}{pathlib.Path(lp).suffix}"
        uri = _upload(lp, dst_key)

        qs = _get_question_list(ex)
        ans = _get_answer_list(ex)

        if not qs and not ans:
            qs = ["Answer the question about the image."]
            ans = [""]

        # Pair questions and answers sensibly
        if len(qs) > 1 and len(ans) > 1:
            k = min(len(qs), len(ans))
            pairs = zip(qs[:k], ans[:k])
        elif len(qs) > 1 and len(ans) == 1:
            pairs = ((q, ans[0]) for q in qs)
        elif len(qs) == 1 and len(ans) > 1:
            pairs = ((qs[0], a) for a in ans)
        else:
            pairs = ((qs[0] if qs else "", ans[0] if ans else ""),)

        for q, a in pairs:
            rows.append((q, a, uri, mime))
            if len(rows) >= want:
                break

    return rows

# -------------------------
# Build & upload JSONL
# -------------------------
def _write_gemini_jsonl(rows: List[Tuple[str, str, str, str]], out_path: str):
    """
    Each line:
    {"contents":[
       {"role":"user","parts":[{"fileData":{"mimeType": "...","fileUri":"gs://..."}},{"text":"question"}]},
       {"role":"model","parts":[{"text":"answer"}]}
    ]}
    """
    with open(out_path, "w", encoding="utf-8") as f:
        for q, a, uri, mime in rows:
            record = {
                "contents": [
                    {"role": "user", "parts": [
                        {"fileData": {"mimeType": mime, "fileUri": uri}},
                        {"text": q or "Answer the question about the image."}
                    ]},
                    {"role": "model", "parts": [{"text": str(a)}]}
                ]
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

# -------------------------
# Run collection, write, and upload
# -------------------------
train_rows = _collect_for_split(TRAIN_SPLIT, MAX_TRAIN_EXAMPLES)
val_rows   = _collect_for_split(VAL_SPLIT,   MAX_VAL_EXAMPLES)

print(f"✅ Prepared {len(train_rows)} train / {len(val_rows)} val examples (uploaded images to GCS).")

train_jsonl_local = "/content/sft_train.jsonl"
val_jsonl_local   = "/content/sft_validation.jsonl"
_ = _write_gemini_jsonl(train_rows, train_jsonl_local)
_ = _write_gemini_jsonl(val_rows,   val_jsonl_local)
print("📄 Wrote:", train_jsonl_local, "and", val_jsonl_local)

def gs_uri(*parts):
    return "gs://" + "/".join([GCS_BUCKET] + [p.strip("/") for p in parts])

# Keep directory names aligned with actual split names
train_jsonl_gcs = gs_uri(GCS_PREFIX, SELECTED_HF_NAME, TRAIN_SPLIT, "sft_train.jsonl")
val_jsonl_gcs   = gs_uri(GCS_PREFIX, SELECTED_HF_NAME, VAL_SPLIT,   "sft_validation.jsonl")

_bucket.blob(train_jsonl_gcs.replace(f"gs://{GCS_BUCKET}/","")).upload_from_filename(train_jsonl_local)
_bucket.blob(val_jsonl_gcs.replace(f"gs://{GCS_BUCKET}/","")).upload_from_filename(val_jsonl_local)

print("🟢 Training JSONL:", train_jsonl_gcs)
print("🟢 Validation JSONL:", val_jsonl_gcs)


Selected HF dataset: dataset
Splits: train='train'  val='validation'  (available: ['train', 'validation', 'test'])
Caps: MAX_TRAIN_EXAMPLES=15  MAX_VAL_EXAMPLES=10
GCS: gs://my-gemini-sft-bucket-group-6/multimodal_sft_20251130_040559/...


Collect validation: 100%|██████████| 10/10 [00:12<00:00,  1.21s/it]


✅ Prepared 15 train / 10 val examples (uploaded images to GCS).
📄 Wrote: /content/sft_train.jsonl and /content/sft_validation.jsonl
🟢 Training JSONL: gs://my-gemini-sft-bucket-group-6/multimodal_sft_20251130_040559/dataset/train/sft_train.jsonl
🟢 Validation JSONL: gs://my-gemini-sft-bucket-group-6/multimodal_sft_20251130_040559/dataset/validation/sft_validation.jsonl


# STEP 6: Start the Gemini supervised fine-tuning job

In [ ]:
#STEP 6 — Start the Gemini supervised fine-tuning job (handles SDK arg name changes)
from vertexai.tuning import sft

BASE_MODEL_ = BASE_MODEL
EPOCHS_     = int(EPOCHS)
LRM_        = float(LEARNING_RATE_MULTIPLIER)
ADAPTER_    = ADAPTER_SIZE_NUM


print("Adapter size is ", ADAPTER_SIZE_NUM)

def launch_sft():
    # We’ll try several argument name variants to handle SDK differences.
    base = {
        "source_model": BASE_MODEL_,
        "train_dataset": train_jsonl_gcs,
        "adapter_size": ADAPTER_,
    }
    # possible variants by SDK version:
    epoch_keys = ["epochs", "epoch_count", "num_epochs"]
    val_keys   = ["validation_dataset", "eval_dataset", "validation_data", None]
    lr_keys    = ["learning_rate_multiplier", "learning_rate", None]

    last_err = None
    for ek in epoch_keys:
        for vk in val_keys:
            for lk in lr_keys:
                kwargs = base.copy()
                kwargs[ek] = EPOCHS_
                if vk: kwargs[vk] = val_jsonl_gcs
                if lk: kwargs[lk] = LRM_
                try:
                    print(f"Trying signature: {ek} / {vk or '(no-val-key)'} / {lk or '(no-lr-key)'}")
                    job = sft.train(**kwargs)
                    print(f"✅ Launched with ({ek}, {vk or '—'}, {lk or '—'})")
                    return job
                except TypeError as e:
                    # wrong signature for this SDK version — try next combo
                    last_err = e
                except Exception as e:
                    # real runtime error (e.g., bad dataset URI) — surface immediately
                    raise
    raise TypeError(f"Could not find a working sft.train signature for this SDK.\nLast error: {last_err}")

tuning_job = launch_sft()

print("🚀 Launched tuning job:")
print("   name:", tuning_job.name)  # projects/.../locations/.../tuningJobs/...
# If you still have `tuning_job` in memory, this will work:
job_id_or_name = tuning_job.name if 'tuning_job' in globals() else "3450221531811020800"  # ← paste ID if needed
globals()['job_id_or_name'] = job_id_or_name  # now available to later

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob


Adapter size is  4
Trying signature: epochs / validation_dataset / learning_rate_multiplier


INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/939190881999/locations/us-central1/tuningJobs/5691505994245341184
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/939190881999/locations/us-central1/tuningJobs/5691505994245341184')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/5691505994245341184?project=939190881999


✅ Launched with (epochs, validation_dataset, learning_rate_multiplier)
🚀 Launched tuning job:
   name: 5691505994245341184


In [ ]:
globals()['job_id_or_name'] = ""  # now available to later

## STEP 6.1 State status of training

In Vertex AI (the service backing Gemini fine-tuning), the state field is an enum.

For SupervisedTuningJob the values map like this:

1 → STATE_UNSPECIFIED (default / not set)

2 → PENDING (job created, not yet running)

3 → RUNNING 🚀 (your training job is actively running right now)

4 → SUCCEEDED (finished successfully)

5 → FAILED

6 → CANCELLING

7 → CANCELLED

In [ ]:
#STEP 6.1 — Inspect a specific tuning job (status, tuned endpoint)
from vertexai.tuning import sft


def _full_job_name(j):
    # Accept either just the numeric ID or the full resource name
    if "/" in j:
        return j
    return f"projects/{PROJECT_ID}/locations/{REGION}/tuningJobs/{j}"

name = _full_job_name(job_id_or_name)
job = sft.SupervisedTuningJob(name).refresh()

print("Job name:   ", job.name)
print("State:      ", job.state)
print("Create time:", getattr(job, "create_time", None))
print("Start time: ", getattr(job, "start_time", None))
print("End time:   ", getattr(job, "end_time", None))

# If finished, a tuned model reference is usually attached:
tm = getattr(job, "tuned_model", None)
if tm:
    print("Tuned model:", tm)
    # Some SDK versions expose an endpoint under tuned_model.endpoint
    if hasattr(tm, "endpoint"):
        print("Endpoint:   ", tm.endpoint)
else:
    print("Tuned model: (not available yet — job is still running)")


Job name:    5691505994245341184
State:       4
Create time: 2025-11-30 04:08:32.651800+00:00
Start time:  None
End time:    None
Tuned model: (not available yet — job is still running)


# STEP 7 : Inspect job & list all jobs

In [ ]:
#STEP 7 — Inspect this tuning job & list all jobs
from vertexai.tuning import sft

assert 'job_id_or_name' in globals(), "Run STEP 6 first to set job_id_or_name."

def _full_job_name(j):
    return j if "/" in j else f"projects/{PROJECT_ID}/locations/{REGION}/tuningJobs/{j}"

name = _full_job_name(job_id_or_name)
job  = sft.SupervisedTuningJob(name).refresh()

print("Job:", job.name)
print("State:", job.state)
print("Create:", getattr(job, "create_time", None))
print("Start :", getattr(job, "start_time", None))
print("End   :", getattr(job, "end_time", None))
tm = getattr(job, "tuned_model", None)
print("Tuned model:", tm if tm else "(not ready yet)")

print("\nAll jobs in project/region:")
for j in sft.SupervisedTuningJob.list():
    print(" •", j.name, "→", j.state, getattr(j, "tuned_model", None))


Job: 5691505994245341184
State: 3
Create: 2025-11-30 04:08:32.651800+00:00
Start : None
End   : None
Tuned model: (not ready yet)

All jobs in project/region:
 • 5691505994245341184 → 3 None
 • 7582676336301309952 → 4 None
 • 22912671271616512 → 4 None


# STEP 8: Fetch tuned endpoint & try a few predictions

In [ ]:
#STEP 8 — Fetch tuned endpoint (Vertex AI client) and test + DISPLAY IMAGES
from google import genai
from google.genai.types import HttpOptions
from vertexai.tuning import sft
from google.cloud import storage
from IPython.display import display
from PIL import Image
import io, requests, re, pathlib

assert 'job_id_or_name' in globals(), "Run STEP 6 first."

# Build full job name if user only saved the numeric ID
name = job_id_or_name if "/" in job_id_or_name else f"projects/{PROJECT_ID}/locations/{REGION}/tuningJobs/{job_id_or_name}"

# ✅ Init the Google GenAI client for **Vertex AI** (Cloud)
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=REGION,
    http_options=HttpOptions(api_version="v1"),
)

# Get tuned endpoint (SDK first, then Tunings API)
job = sft.SupervisedTuningJob(name).refresh()
print("Job state:", job.state)

endpoint = None
tm = getattr(job, "tuned_model", None)
if tm and getattr(tm, "endpoint", None):
    endpoint = tm.endpoint
if endpoint is None:
    tuned = client.tunings.get(name=name)
    if getattr(tuned, "tuned_model", None):
        endpoint = tuned.tuned_model.endpoint

if not endpoint:
    raise RuntimeError("Tuned endpoint not available yet. Confirm the job has SUCCEEDED (see STEP 7/7.1 or Console).")

print("🟢 Tuned endpoint:", endpoint)
globals()['endpoint'] = endpoint  # expose for later steps

# ---------- helpers to load & display images ----------
_storage = storage.Client(project=PROJECT_ID)

def _load_image(uri_or_path):
    """Return a PIL.Image from gs://, http(s)://, or local path."""
    if isinstance(uri_or_path, str) and uri_or_path.startswith("gs://"):
        # gs://bucket/key
        m = re.match(r"gs://([^/]+)/(.+)", uri_or_path)
        if not m:
            return None
        bkt, key = m.group(1), m.group(2)
        blob = _storage.bucket(bkt).blob(key)
        data = blob.download_as_bytes()
        return Image.open(io.BytesIO(data)).convert("RGB")
    if isinstance(uri_or_path, str) and uri_or_path.startswith(("http://","https://")):
        r = requests.get(uri_or_path, timeout=30)
        r.raise_for_status()
        return Image.open(io.BytesIO(r.content)).convert("RGB")
    # local file path (rare in this flow)
    p = pathlib.Path(str(uri_or_path))
    if p.exists():
        return Image.open(p).convert("RGB")
    return None

def _safe_text(resp):
    try:
        return (resp.text or "").strip()
    except Exception:
        parts = getattr(resp, "candidates", []) or []
        for c in parts:
            content = getattr(c, "content", None)
            if content:
                for part in getattr(content, "parts", []) or []:
                    if isinstance(part, dict) and "text" in part:
                        return part["text"].strip()
        return ""

# ---- Test a few validation examples & DISPLAY the image ----
assert 'val_rows' in globals() and len(val_rows), "Run STEP 5 to prepare val_rows."
NUM_PREVIEW = 5  # change if you want more/less
preview = val_rows[:min(NUM_PREVIEW, len(val_rows))]

print(f"Testing {len(preview)} validation samples (showing images)…")

for i, (q, a, uri, mime) in enumerate(preview, start=1):
    print(f"\n[{i}] Q:", q)
    img = _load_image(uri)
    if img:
        display(img)
    else:
        print("(Image preview unavailable)")

    prompt = [
        {"role":"user","parts":[
            {"file_data": {"file_uri": uri, "mime_type": mime}},
            {"text": q or "Answer the question about the image."}
        ]}
    ]
    resp = client.models.generate_content(model=endpoint, contents=prompt)
    pred = _safe_text(resp)

    print("GT:", a)
    print("Pred:", pred)


#STEP 9: Compare Fine-Tuned Model to Original Gemini

### STEP 9.1 run predictions on test data for both fine-tuned & base Gemini model

In [ ]:
# Step 9.1 A) — Evaluate tuned Gemini vs base Gemini using SFT JSONL (reads GT from file)
from typing import Any, Dict, List, Optional, Tuple
import json, os
from tqdm import tqdm
from google.genai import errors as genai_errors

# Reuse client/endpoint from STEP 8
assert 'client' in globals(), "Run STEP 8 first to create `client`."
assert 'endpoint' in globals() and endpoint, "Run STEP 8 first to resolve `endpoint`."

# ---------- Resolve a valid base Gemini model (prefer Gemini 2.5 Flash) ----------
def _probe_model_id(mid: str) -> bool:
    """Return True if this model id is available to this project/region."""
    try:
        try:
            client.models.get(mid)            # most SDKs accept positional
        except TypeError:
            client.models.get(model=mid)      # fallback signature
        return True
    except genai_errors.ClientError as e:
        # 404 => not available; 403 => not permitted; 400 => bad id format
        if getattr(e, "status_code", None) not in (404, 403, 400):
            print(f"Probe {mid} -> {getattr(e,'status_code',None)} {getattr(e,'message',e)}")
        return False

def _resolve_base_gemini(prefer_25: bool = True) -> str:
    candidates_25 = [
        "publishers/google/models/gemini-2.5-flash",
        "publishers/google/models/gemini-2.5-flash-001",
        "models/gemini-2.5-flash",
        "models/gemini-2.5-flash-001",
    ]
    candidates_15 = [
        "publishers/google/models/gemini-1.5-flash-001",
        "publishers/google/models/gemini-1.5-pro-002",
        "models/gemini-1.5-flash-001",
        "models/gemini-1.5-pro-002",
    ]
    pool = (candidates_25 if prefer_25 else []) + candidates_15
    for mid in pool:
        if _probe_model_id(mid):
            print("✅ Base model available:", mid)
            return mid
    raise RuntimeError("No base Gemini model ID worked in this project/region.")

# Respect an existing selection; else honor user BASE_MODEL; else resolve automatically
if 'BASE_MODEL_FOR_GENAI' not in globals() or not BASE_MODEL_FOR_GENAI:
    user_base = globals().get("BASE_MODEL", None)
    if user_base:
        candidate = user_base if user_base.startswith(("models/","publishers/")) else f"models/{user_base}"
        if _probe_model_id(candidate):
            BASE_MODEL_FOR_GENAI = candidate
            print("Base model    :", BASE_MODEL_FOR_GENAI)
        else:
            print(f"Provided BASE_MODEL '{candidate}' not available; resolving a valid one…")
            BASE_MODEL_FOR_GENAI = _resolve_base_gemini(prefer_25=True)
    else:
        BASE_MODEL_FOR_GENAI = _resolve_base_gemini(prefer_25=True)

print("Tuned endpoint:", endpoint)
print("Base model    :", BASE_MODEL_FOR_GENAI)

# ---------- Use the SFT validation file (contains question + ground-truth answer) ----------
test_path = "sft_validation.jsonl"  # adjust path if needed
assert os.path.exists(test_path), f"Missing {test_path}"

# ---------- Helpers ----------
def _guess_mime(uri: str) -> str:
    u = (uri or "").lower()
    if u.endswith(".png"):  return "image/png"
    if u.endswith(".webp"): return "image/webp"
    if u.endswith(".gif"):  return "image/gif"
    if u.endswith(".bmp"):  return "image/bmp"
    return "image/jpeg"

def parse_sft_record(rec: Dict[str, Any]) -> Tuple[str, Optional[str], Optional[str], str]:
    """
    Parse a Gemini SFT JSONL record like:
      {"contents":[
         {"role":"user","parts":[{"fileData":{"mimeType":"...","fileUri":"gs://..."}},{"text":"question"}]},
         {"role":"model","parts":[{"text":"answer"}]}
      ]}
    Returns: (question_text, image_uri, mime_type, ground_truth_answer)
    """
    q_text, img_uri, mime_type, gt = "", None, None, ""
    contents = rec.get("contents", [])
    for message in contents:
        role = message.get("role")
        for part in (message.get("parts") or []):
            if isinstance(part, dict):
                if role == "user" and "text" in part and not q_text:
                    q_text = str(part["text"])
                fd = part.get("fileData") or part.get("file_data")
                if role == "user" and isinstance(fd, dict):
                    img_uri   = fd.get("fileUri")  or fd.get("file_uri")  or img_uri
                    mime_type = fd.get("mimeType") or fd.get("mime_type") or mime_type
                if role == "model" and "text" in part and not gt:
                    gt = str(part["text"])
    return q_text.strip(), img_uri, mime_type, gt.strip()

def parse_messages_record(rec: Dict[str, Any]) -> Tuple[str, Optional[str], Optional[str], str]:
    # Fallback if you ever point test_path to an OpenAI-style messages file
    msgs = rec.get("messages", [])
    user_msgs = [m for m in msgs if m.get("role") == "user"]
    last = user_msgs[-1] if user_msgs else (msgs[-1] if msgs else {})
    txt = ""
    img_uri, mime_type = None, None
    content = last.get("content", [])
    if isinstance(content, dict): content = [content]
    for c in content or []:
        if c.get("type") == "text" and "text" in c:
            txt += c["text"] if isinstance(c["text"], str) else str(c["text"])
        if c.get("type") in ("input_image", "image_url", "input_image_url", "image"):
            if isinstance(c.get("image_url"), dict):
                img_uri = c["image_url"].get("url", img_uri)
            elif isinstance(c.get("image_url"), str):
                img_uri = c["image_url"]
            elif "url" in c:
                img_uri = c["url"]
    if not txt and isinstance(last.get("content"), str):
        txt = last["content"]
    mime_type = _guess_mime(img_uri) if img_uri and not mime_type else mime_type
    return txt.strip(), img_uri, mime_type, ""

def parse_record(rec: Dict[str, Any]) -> Tuple[str, Optional[str], Optional[str], str]:
    return parse_sft_record(rec) if "contents" in rec else parse_messages_record(rec)

def genai_predict(model_name: str, q_text: str, img_uri: Optional[str], mime_type: Optional[str]) -> str:
    try:
        parts = []
        if img_uri:
            parts.append({"file_data": {"file_uri": img_uri, "mime_type": mime_type or _guess_mime(img_uri)}})
        parts.append({"text": q_text or "Answer the question about the image."})
        prompt = [{"role": "user", "parts": parts}]
        resp = client.models.generate_content(model=model_name, contents=prompt)
        txt = getattr(resp, "text", None)
        if txt: return txt.strip()
        cands = getattr(resp, "candidates", []) or []
        if cands:
            content = getattr(cands[0], "content", None)
            if content and getattr(content, "parts", None):
                first = content.parts[0]
                if isinstance(first, dict) and "text" in first:
                    return str(first["text"]).strip()
        return ""
    except Exception as e:
        return f"[ERROR] {type(e).__name__}: {e}"

# ---------- Load tests ----------
test_data: List[Dict[str, Any]] = []
with open(test_path, "r", encoding="utf-8") as f:
    for line in f:
        test_data.append(json.loads(line))

# ---------- Evaluate ----------
def evaluate_model(model_name: str, out_path: str) -> int:
    results = []
    for idx, rec in tqdm(enumerate(test_data), total=len(test_data), desc=f"Eval {os.path.basename(out_path)}"):
        q_text, img_uri, mime_type, gt = parse_record(rec)
        pred = genai_predict(model_name, q_text, img_uri, mime_type)
        results.append({
            "example_id": idx,               # index in the file
            "predicted_answer": pred,
            "actual_answer": gt              # ground truth directly from file
        })
    with open(out_path, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    return len(results)

ft_out   = "ocr-vqa-gemini-ft-results.jsonl"
base_out = "ocr-vqa-gemini-base-results.jsonl"

n1 = evaluate_model(endpoint, ft_out)
print(f"✓ wrote {ft_out} ({n1} rows)")
n2 = evaluate_model(BASE_MODEL_FOR_GENAI, base_out)
print(f"✓ wrote {base_out} ({n2} rows)")


Probe models/gemini-2.5-flash -> None List of found errors:	1.Field: name; Message: Invalid Model resource name.	
Provided BASE_MODEL 'models/gemini-2.5-flash' not available; resolving a valid one…
✅ Base model available: publishers/google/models/gemini-2.5-flash
Tuned endpoint: projects/939190881999/locations/us-central1/endpoints/4594470964871299072
Base model    : publishers/google/models/gemini-2.5-flash


Eval ocr-vqa-gemini-ft-results.jsonl: 100%|██████████| 10/10 [02:13<00:00, 13.32s/it]


✓ wrote ocr-vqa-gemini-ft-results.jsonl (10 rows)


Eval ocr-vqa-gemini-base-results.jsonl: 100%|██████████| 10/10 [01:57<00:00, 11.74s/it]

✓ wrote ocr-vqa-gemini-base-results.jsonl (10 rows)


In [ ]:
# Step 9.1 B) — Preview first eval sample (image + Q/A + predictions)

import os, json, io
from IPython.display import display
from PIL import Image as PILImage

# Paths from Step 9.1 (change if you renamed them)
test_path = "sft_validation.jsonl"
ft_out    = "ocr-vqa-gemini-ft-results.jsonl"
base_out  = "ocr-vqa-gemini-base-results.jsonl"

def _guess_mime(uri: str) -> str:
    u = (uri or "").lower()
    if u.endswith(".png"):  return "image/png"
    if u.endswith(".webp"): return "image/webp"
    if u.endswith(".gif"):  return "image/gif"
    if u.endswith(".bmp"):  return "image/bmp"
    return "image/jpeg"

def _parse_first_sft(path: str):
    """Return (question, image_uri, mime_type, ground_truth) for the first record."""
    with open(path, "r", encoding="utf-8") as f:
        line = f.readline()
    rec = json.loads(line)

    q_text, img_uri, mime_type, gt = "", None, None, ""
    contents = rec.get("contents", [])
    for message in contents:
        role = message.get("role")
        for part in (message.get("parts") or []):
            if isinstance(part, dict):
                if role == "user" and "text" in part and not q_text:
                    q_text = str(part["text"])
                fd = part.get("fileData") or part.get("file_data")
                if role == "user" and isinstance(fd, dict) and not img_uri:
                    img_uri   = fd.get("fileUri")  or fd.get("file_uri")
                    mime_type = fd.get("mimeType") or fd.get("mime_type")
                if role == "model" and "text" in part and not gt:
                    gt = str(part["text"])
    if img_uri and not mime_type:
        mime_type = _guess_mime(img_uri)
    return q_text.strip(), img_uri, mime_type, gt.strip()

def _load_prediction(jsonl_path: str, idx: int = 0):
    """Read 'predicted_answer' for row idx from a JSONL results file."""
    if not os.path.exists(jsonl_path):
        return None
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i == idx:
                try:
                    return json.loads(line).get("predicted_answer", "")
                except Exception:
                    return ""
    return None

def _fetch_image(uri: str):
    """Return a PIL Image from gs://, http(s)://, or local path."""
    if not uri:
        return None
    if uri.startswith("gs://"):
        from google.cloud import storage
        bucket_name, blob_name = uri[5:].split("/", 1)
        tmp_path = f"/tmp/_preview_{os.path.basename(blob_name)}"
        storage.Client().bucket(bucket_name).blob(blob_name).download_to_filename(tmp_path)
        return PILImage.open(tmp_path)
    if uri.startswith("http"):
        import requests
        resp = requests.get(uri, timeout=30)
        resp.raise_for_status()
        return PILImage.open(io.BytesIO(resp.content))
    # assume local file path
    return PILImage.open(uri)

# --- Grab the first sample ---
q_text, img_uri, mime_type, gt = _parse_first_sft(test_path)
tuned_pred = _load_prediction(ft_out, 0)
base_pred  = _load_prediction(base_out, 0)

print("Question:", q_text or "(none)")
print("Image URI:", img_uri or "(none)")
print("Ground truth:", gt or "(none)")
if tuned_pred is not None:
    print("Tuned prediction:", tuned_pred)
if base_pred is not None:
    print("Base prediction :", base_pred)

img = _fetch_image(img_uri) if img_uri else None
if img:
    display(img)
else:
    print("(No image available to display)")


### STEP 9.2 Display results in table - side by side

In [ ]:
# STEP 9.2 — Compare tuned-vs-base predictions on the first 10 examples
import pandas as pd
import json

ft_out   = "ocr-vqa-gemini-ft-results.jsonl"
base_out = "ocr-vqa-gemini-base-results.jsonl"

# Load results
def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    return rows

ft_rows   = load_jsonl(ft_out)
base_rows = load_jsonl(base_out)

ft_df   = pd.DataFrame(ft_rows)
base_df = pd.DataFrame(base_rows)

# Merge on example_id to align predictions
merged = pd.merge(ft_df, base_df, on="example_id", suffixes=("_ft", "_base"))

# Select top 10 to preview
top10 = merged.head(10)[[
    "example_id",
    "predicted_answer_ft",
    "predicted_answer_base",
    "actual_answer_ft"    # same ground truth; pulled from ft file
]].rename(columns={
    "predicted_answer_ft":   "Fine-tuned Prediction",
    "predicted_answer_base": "Base Gemini Prediction",
    "actual_answer_ft":      "Actual Answer"
})

from IPython.display import display
display(top10)
